In [45]:
import fasttext
import json
from faq50_adapted import FAQ, extract_word_probs
from topic_word_probs import *
from tfidf_classifier import TFIDF_Classifier
from cs_lemmatizer import *


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
model_path = "models/cc.cs.300.bin"
model = fasttext.load_model(model_path)

### Tests:

In [80]:
path_to_corpus = "upv_faq/Q78_questions.xlsx"
path_to_save = "780_upv_questions/w_probs_in_questions.json"
count_word_probs_in_questions(path_to_corpus, path_to_save)
with open(path_to_save, "r") as wp_file:
    probs_q = json.load(wp_file)


path_to_corpus = "upv_faq/Q78_answers_no_tags.xlsx"
path_to_save = "780_upv_questions/w_probs_in_answers.json"
count_word_probs_in_answers(path_to_corpus, path_to_save)
with open(path_to_save, "r") as wp_file:
    probs_a = json.load(wp_file)


path_to_save = "780_upv_questions/w_probs_in_q_and_a.json"
probs = combine_dicts(probs_a, probs_q, save_path=path_to_save)

# print(dict(sorted(probs.items(), key=lambda item: -item[1])))

In [81]:
q_xslx = "upv_faq/Q78_questions.xlsx"
a_xslx = "upv_faq/Q78_answers.xlsx"

faq = FAQ(model, q_xslx, a_xslx, probs=probs, alpha=0.1)

In [82]:
acc, acc_sec = faq.cross_match_test()
print(f"\nQuestion Cross-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")

acc, acc_sec = faq.mean_match_test()
print(f"\nQuestion Mean-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")


Question Cross-Match Accuracy: 1st: 0.462, 2nd: 0.374

Question Mean-Match Accuracy: 1st: 0.695, 2nd: 0.127


### Mean match test without inclusion of tested question:

In [83]:
f, s, t = faq.mean_match_test_disjunctive()
print(f"\033[1m1st right: {f} \033[0m")
print(f"2nd right: {s}\n3rd right: {t}\n-> {round(f+s+t, 3)}")

1st right: 0.48 
2nd right: 0.138
3rd right: 0.067
-> 0.685


In [55]:
LMTZR.remove_stop_words_from_sentence(["rešerše", "mezi", "jsem", "přihlasit", "jak", "patentem"])

['rešerše', 'přihlasit', 'patentem']

## Confused sentences:

In [84]:
# Ambiguous matches
path_to_save = "780_upv_questions/same_question_different_answers.json"
faq.get_same_question_different_answer_pairs(save_path=path_to_save)

Same question with different classes:

Q: Jaké jsou hlavní rozdíly mezi užitným vzorem a patentem? It's idx= 341 , true_class= 34
Q_argmax: Jaký je hlavní rozdíl mezi patentem a užitným vzorem? It's idx=  329 , true_class= 33

Q: Jaké jsou možnosti podání přihlášky průmyslového vzoru?  It's idx= 512 , true_class= 51
Q_argmax: Jaké jsou možnosti podání přihlášky průmyslového vzoru? It's idx=  490 , true_class= 49

Q: Kdo může podat přihlášku průmyslového vzoru? It's idx= 519 , true_class= 52
Q_argmax: Kdo může podat přihlášku průmyslového vzoru? It's idx=  492 , true_class= 49

Q: Jak mohu objednat rešeršní služby na Úřadu? It's idx= 550 , true_class= 55
Q_argmax: Jak mohu objednat rešeršní služby Úřadu? It's idx=  449 , true_class= 45

Q: Kdo může podat přihlášku užitného vzoru? It's idx= 592 , true_class= 59
Q_argmax: Jak mohu podat přihlášku užitného vzoru? It's idx=  589 , true_class= 59

Q: Kdo může podat přihlášku užitného vzoru? It's idx= 609 , true_class= 61
Q_argmax: Jak mohu p

{'Jaké jsou hlavní rozdíly mezi užitným vzorem a patentem?': ['34', '33'],
 'Jaké jsou možnosti podání přihlášky průmyslového vzoru? ': ['51', '49'],
 'Kdo může podat přihlášku průmyslového vzoru?': ['52', '49'],
 'Jak mohu objednat rešeršní služby na Úřadu?': ['55', '45'],
 'Kdo může podat přihlášku užitného vzoru?': ['59', '61'],
 'Jak mohu objednat rešeršní služby Úřadu?': ['65', '45'],
 'Jak dlouho trvá zpracování rešerše?': ['65', '45', '74'],
 'Co znamená kvalifikované zveřejnění technického řešení?': ['68', '58'],
 'Jakým způsobem lze podat objednávku na rešeršní službu?': ['74', '55']}

In [155]:
faq.get_most_confused_questions(cos_sim_threshold=0.85) # with mean-match

(788, 79)
(788,)
Series([], Name: question, dtype: object)


In [156]:
path_to_save = "780_upv_questions/most_missclassified_class_pairs"
faq.get_most_misclassified_class_pairs(n_of_common_misses=3, save_path=path_to_save)

(0, 66) 4
(19, 55) 3
(19, 42) 3
(22, 7) 3
(30, 55) 4
(31, 69) 3
(34, 33) 3
(36, 57) 5
(38, 39) 3
(40, 4) 3
(45, 55) 3
(46, 0) 3
(53, 62) 6
(55, 74) 3
(56, 0) 3
(58, 68) 7
(65, 55) 3
(72, 14) 4
(77, 3) 5
['0:66', '19:55', '19:42', '22:7', '30:55', '31:69', '34:33', '36:57', '38:39', '40:4', '45:55', '46:0', '53:62', '55:74', '56:0', '58:68', '65:55', '72:14', '77:3']


In [157]:
questions_xlsx = "780_upv_questions/expanded_data_all2.xlsx"
# questions_xlsx = "upv_faq/Q50_questions.xlsx"
c = TFIDF_Classifier(questions_xlsx)

In [158]:
test_data = c.structure_data(test_data_percent=0.1) 

c.get_TFIDF_matrix()

TFIDF matrix shape: (79, 809)


In [159]:
c.classify_sentence("Co by nastalo kdyby patent byl zrušen?")

73

In [160]:
c.classify_test_sentences_list(test_data)

Got right: 0.41025641025641024


In [161]:
test_data = c.structure_data(test_data_percent=1) # without removal from train data
c.get_TFIDF_matrix()
c.classify_test_sentences_list(test_data)

TFIDF matrix shape: (79, 848)
Got right: 0.8439086294416244
